# Algorytm k najbliższych sąsiadów

W ćwiczeniu wykorzystywany jest zbiór danych `donors.csv`, opracowany przez organizację weteranów, która zabiga o datki, kierując kampanie listowe do aktualnych i potencjalnych darczyńców z bazy danych. Zbiór danych zawiera informacje o cechach charakterystycznych klientów oraz o tym, czy zareagowali na kampanię testową. Zadaniem jest pomoc organizacji w przewidzeniu, którzy darczyńcy odpowiedzą na kampanię listową w oparciu o dane demograficzne, wcześniejszą historię datków oraz reakcję na poprzednie kampanie listowe.

1. Zaimportuj dane i wyświetl ich podgląd (funkcje `read_csv` oraz `glimpse`. Pierwsze 12 zmiennych jest typu `numeric`, pozostałe 10 jest typu `factor`). Cecha klasy nosi nazwę `respondentMailing`.

In [1]:
library(tidyverse)
options(jupyter.rich_display = FALSE,
        repr.plot.width = 15,
        repr.plot.height = 8)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
df <- read_csv("data/donors.csv")

Rows: 95412 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): state, urbanicity, socioEconomicStatus, gender
dbl (12): age, numberChildren, incomeRating, wealthRating, mailOrderPurchase...
lgl  (6): inHouseDonor, plannedGivingDonor, sweepstakesDonor, P3Donor, isHom...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
head(df)

  age numberChildren incomeRating wealthRating mailOrderPurchases
1 60  NA             NA           NA            0                
2 46   1              6            9           16                
3 NA  NA              3            1            2                
4 70  NA              1            4            2                
5 78   1              3            2           60                
6 NA  NA             NA           NA            0                
  totalGivingAmount numberGifts smallestGiftAmount largestGiftAmount
1 240               31           5                 12               
2  47                3          10                 25               
3 202               27           2                 16               
4 109               16           2                 11               
5 254               37           3                 15               
6  51                4          10                 16               
  averageGiftAmount ⋯ inHouseDonor plannedGivingDonor s

In [4]:
dim(df)

[1] 95412    22

In [5]:
glimpse(df)

Rows: 95,412
Columns: 22
$ age                     <dbl> 60, 46, NA, 70, 78, NA, 38, NA, NA, 65, NA, 75…
$ numberChildren          <dbl> NA, 1, NA, NA, 1, NA, 1, NA, NA, NA, NA, NA, 2…
$ incomeRating            <dbl> NA, 6, 3, 1, 3, NA, 4, 2, 3, NA, 2, 1, 4, NA, …
$ wealthRating            <dbl> NA, 9, 1, 4, 2, NA, 6, 9, 2, NA, 0, 5, 2, NA, …
$ mailOrderPurchases      <dbl> 0, 16, 2, 2, 60, 0, 0, 1, 0, 0, 0, 3, 16, 0, 1…
$ totalGivingAmount       <dbl> 240, 47, 202, 109, 254, 51, 107, 31, 199, 28, …
$ numberGifts             <dbl> 31, 3, 27, 16, 37, 4, 14, 5, 11, 3, 1, 2, 9, 1…
$ smallestGiftAmount      <dbl> 5, 10, 2, 2, 3, 10, 3, 5, 10, 3, 20, 10, 4, 5,…
$ largestGiftAmount       <dbl> 12, 25, 16, 11, 15, 16, 12, 11, 22, 15, 20, 15…
$ averageGiftAmount       <dbl> 7.741935, 15.666667, 7.481481, 6.812500, 6.864…
$ yearsSinceFirstDonation <dbl> 8, 3, 7, 10, 11, 3, 10, 3, 9, 3, 1, 1, 8, 5, 4…
$ monthsSinceLastDonation <dbl> 14, 14, 14, 14, 13, 20, 22, 18, 19, 22, 12, 14…
$ inHouseDonor 

In [6]:
df <- df %>%
  mutate(
         age = as.integer(age),
         numberChildren = as.integer(numberChildren),
         incomeRating = as.integer(incomeRating),
         wealthRating = as.integer(wealthRating),
         mailOrderPurchases = as.integer(mailOrderPurchases),
         totalGivingAmount = as.integer(totalGivingAmount),
         numberGifts = as.integer(numberGifts),
         smallestGiftAmount = as.integer(smallestGiftAmount),
         largestGiftAmount = as.integer(largestGiftAmount),
         averageGiftAmount = as.double(averageGiftAmount),
         yearsSinceFirstDonation = as.integer(yearsSinceFirstDonation),
         monthsSinceLastDonation = as.integer(monthsSinceLastDonation),
         inHouseDonor = as.factor(inHouseDonor),
         plannedGivingDonor = as.factor(plannedGivingDonor),
         sweepstakesDonor = as.factor(sweepstakesDonor),
         P3Donor = as.factor(P3Donor),
         state = as.factor(state),
         urbanicity = as.factor(urbanicity),
         socioEconomicStatus = as.factor(socioEconomicStatus),
         isHomeowner = as.factor(isHomeowner),
         gender = as.factor(gender), 
         respondedMailing = as.factor(respondedMailing))

In [7]:
glimpse(df)

Rows: 95,412
Columns: 22
$ age                     <int> 60, 46, NA, 70, 78, NA, 38, NA, NA, 65, NA, 75…
$ numberChildren          <int> NA, 1, NA, NA, 1, NA, 1, NA, NA, NA, NA, NA, 2…
$ incomeRating            <int> NA, 6, 3, 1, 3, NA, 4, 2, 3, NA, 2, 1, 4, NA, …
$ wealthRating            <int> NA, 9, 1, 4, 2, NA, 6, 9, 2, NA, 0, 5, 2, NA, …
$ mailOrderPurchases      <int> 0, 16, 2, 2, 60, 0, 0, 1, 0, 0, 0, 3, 16, 0, 1…
$ totalGivingAmount       <int> 240, 47, 202, 109, 254, 51, 107, 31, 199, 28, …
$ numberGifts             <int> 31, 3, 27, 16, 37, 4, 14, 5, 11, 3, 1, 2, 9, 1…
$ smallestGiftAmount      <int> 5, 10, 2, 2, 3, 10, 3, 5, 10, 3, 20, 10, 4, 5,…
$ largestGiftAmount       <int> 12, 25, 16, 11, 15, 16, 12, 11, 22, 15, 20, 15…
$ averageGiftAmount       <dbl> 7.741935, 15.666667, 7.481481, 6.812500, 6.864…
$ yearsSinceFirstDonation <int> 8, 3, 7, 10, 11, 3, 10, 3, 9, 3, 1, 1, 8, 5, 4…
$ monthsSinceLastDonation <int> 14, 14, 14, 14, 13, 20, 22, 18, 19, 22, 12, 14…
$ inHouseDonor 

2. Ogranicz predyktory wyłącznie do cech liczbowych ze zbioru danych (funkcja `select`). Wyświetl podsumowanie statystyczne dla nowego zbioru danych i sprawdź dla których zmiennych występują braki danych.

In [8]:
df_numeric <- df %>%
  select(where(is.numeric))

In [9]:
summary(df_numeric)

      age        numberChildren   incomeRating    wealthRating  
 Min.   : 1.00   Min.   :1.00    Min.   :1.000   Min.   :0.00   
 1st Qu.:48.00   1st Qu.:1.00    1st Qu.:2.000   1st Qu.:3.00   
 Median :62.00   Median :1.00    Median :4.000   Median :6.00   
 Mean   :61.61   Mean   :1.53    Mean   :3.886   Mean   :5.35   
 3rd Qu.:75.00   3rd Qu.:2.00    3rd Qu.:5.000   3rd Qu.:8.00   
 Max.   :98.00   Max.   :7.00    Max.   :7.000   Max.   :9.00   
 NA's   :23665   NA's   :83026   NA's   :21286   NA's   :44732  
 mailOrderPurchases totalGivingAmount  numberGifts      smallestGiftAmount
 Min.   :  0.000    Min.   :  13.0    Min.   :  1.000   Min.   :   0.00   
 1st Qu.:  0.000    1st Qu.:  40.0    1st Qu.:  3.000   1st Qu.:   3.00   
 Median :  0.000    Median :  78.0    Median :  7.000   Median :   5.00   
 Mean   :  3.321    Mean   : 104.5    Mean   :  9.602   Mean   :   7.93   
 3rd Qu.:  3.000    3rd Qu.: 131.0    3rd Qu.: 13.000   3rd Qu.:  10.00   
 Max.   :241.000    Max.   :94

In [10]:
numberofna <- df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

In [11]:
round(numberofna / nrow(df_numeric) * 100, 2)

[1] 91.86

*92% wierszy ma w sobie `NA`*

3. Uzupełnij brakujące dane: dla zmiennej `age` użyj imputacji średniej, dla zmiennej `numberChildren` - imputacji mediany. W przypadku cech `incomeRating` oraz `wealthRating` problem brakujących danych rozwiąż wykluczając te wystąpienia ze zbioru danych. Ponadto dla zmiennej `wealthRating` (miara ogólnego majątku darczyńcy w skali od 1 do 9) należy również wykluczyć wystąpienia o wartości 0.

In [12]:
df_numeric["age"][is.na(df_numeric["age"])] <- as.integer(mean(df_numeric$age, na.rm = TRUE))
df_numeric["numberChildren"][is.na(df_numeric["numberChildren"])] <- median(df_numeric$numberChildren, na.rm = TRUE)

df_numeric <- df_numeric %>%
    filter(!is.na(incomeRating) & 
           !is.na(wealthRating) & 
           wealthRating > 0
    )

In [13]:
df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 0

In [14]:
dim(df_numeric)

[1] 48267    12

*nie ma braków juz, zostało 48_267 wierszy*

4. Utwórz funkcję normalizacji min-max i zapisz ją jako `normalize`. Następnie wykorzystaj ją do znormalizowania wartości cech (ustandaryzowania skali do zakresu od 0 do 1).

In [32]:
min_max <- function(x) {
    return ( (x - min(x) ) / (max(x) - min(x)) )
}

In [34]:
df_numeric_norm <- sapply(df_numeric, function(x) min_max(x))
head(df_numeric_norm)

     age       numberChildren incomeRating wealthRating mailOrderPurchases
[1,] 0.4639175 0              0.8333333    1.000        0.066390041       
[2,] 0.6185567 0              0.3333333    0.000        0.008298755       
[3,] 0.7113402 0              0.0000000    0.375        0.008298755       
[4,] 0.7938144 0              0.3333333    0.125        0.248962656       
[5,] 0.3814433 0              0.5000000    0.625        0.000000000       
[6,] 0.6185567 0              0.1666667    1.000        0.004149378       
     totalGivingAmount numberGifts smallestGiftAmount largestGiftAmount
[1,] 0.006006006       0.008474576 0.020              0.020100503      
[2,] 0.033386328       0.110169492 0.004              0.011055276      
[3,] 0.016958135       0.063559322 0.004              0.006030151      
[4,] 0.042571984       0.152542373 0.006              0.010050251      
[5,] 0.016604840       0.055084746 0.006              0.007035176      
[6,] 0.003179650       0.016949153 0.010   

5. Podziel dane na zbiory treningowy i testowy w stosunku 75:25, przekształcając uprzednio dane w ramkę danych. Porównaj rozkłady klas dla oryginalnego zbioru danych oraz zbiorów treningowego i testowego.

6. Za pomocą funkcji `smote` z pakietu `performanceEstimation` zrównoważ dane treningowe.

In [16]:
library(performanceEstimation)

7. Przenieś etykiety klas do osobnych zbiorów danych. Z pomocą polecenia `pull()` z pakietu `tidyverse` utwórz nowe wektory z etykietami cechy klasy (`respondedMailing`) i przekształć je do typu `factor`.

8. Przekształć zbiory danych treningowych i testowych do postaci ramek danych bez etykiet klas (`respondedMailing`).

9. Budowanie modelu: Wykorzystaj funkcję `knn()` z pakietu `class` do oznaczenia przykładów testowych na podstawie danych treningowych. `k` ustaw wartość `5`.

In [17]:
library(class)



10. Wyświetl pierwsze 6 predykcji.

11. Utwórz macierz pomyłek i na jej podstawie oblicz dokładność predykcji.

12. Zmodyfikuj kod tak, aby użyć zmiennych kategorialnych w celu poprawy dokładności modelu.